<a href="https://colab.research.google.com/github/Bha5vya/Google-Earth-Engine-Basic-Code/blob/main/LULC_Classification_(ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap.geemap as map
import ee
import os

ee.Authenticate()
ee.Initialize(project='bha5vya')

vis_map = map.Map()

In [2]:
rajasthan_boundary = ee.FeatureCollection('projects/bha5vya/assets/Rajasthan')
vis_map.addLayer(rajasthan_boundary, {}, 'Rajasthan Boundary')

In [3]:
value_visual = {
    'min': 10, 'max': 100, 'palette': [
        '#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000',
        '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0'
    ]
}

In [4]:
Legend = {
    'Tree Cover': '#006400',
    'Shrubland': '#ffbb22',
    'Grassland': '#ffff4c',
    'Cropland': '#f096ff',
    'Built-up': '#fa0000',
    'Bare / sparse vegetation': '#b4b4b4',
    'Snow and ice': '#f0f0f0',
    'Permanent water bodies': '#0064c8',
    'Herbaceous wetland': '#0096a0',
    'Mangroves': '#00cf75',
    'Moss and lichen': '#fae6a0'
}

In [5]:
rajasthan_esa = ee.ImageCollection('ESA/WorldCover/v200')
rajasthan_esa_mean = rajasthan_esa.mean().clip(rajasthan_boundary).select('Map')

In [6]:
vis_map.centerObject(rajasthan_esa_mean)
vis_map.addLayer(rajasthan_esa_mean, value_visual, 'Rajasthan ESA Worldcover')
vis_map.add_legend(title="Land Use/Land Cover Legend", legend_dict=Legend, **value_visual)

In [7]:
landsat8_Data = ee.ImageCollection("LANDSAT/LC08/C02/T1")\
    .filterDate('2024-01-01', '2024-12-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 5)\
    .mean().clip(rajasthan_boundary)

In [8]:
vis_map.addLayer(landsat8_Data, {}, 'Landsat8_2024')

In [9]:
TP = rajasthan_esa_mean.sample(
    region=rajasthan_boundary.geometry(),
    scale=20,
    numPixels=4000,
    seed=1,
    geometries=True
)

vis_map.addLayer(TP, {}, 'Training Points')

In [10]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6']
label = 'Map'

In [11]:
training = landsat8_Data.select(bands).sampleRegions(
    collection=TP,
    properties=[label],
    scale=30
)

In [12]:
trained = ee.Classifier.smileKNN().train(training, label, bands)
result = landsat8_Data.select(bands).classify(trained)
vis_map.addLayer(result, value_visual, 'Supervised Classification')

In [13]:
vis_map

Map(center=[0.0, 0.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…